In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

data_dir = '/home/maith/Desktop/practical_2/cityscapes'
train_images_dir = os.path.join(data_dir, 'leftImg8bit/train')
train_labels_dir = os.path.join(data_dir, 'gtFine/train')

def preprocess_image(image_path, label_path, target_size=(256, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label)
    return image, tf.cast(label, tf.int32)

def augment(image, label):
    label = tf.expand_dims(label, axis=-1)
    
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        label = tf.image.flip_left_right(label)

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)

    label = tf.squeeze(label, axis=-1)

    return image, label

def file_paths_generator():
    count = 0
    for city in sorted(os.listdir(train_images_dir)):
        city_images_path = os.path.join(train_images_dir, city)
        city_labels_path = os.path.join(train_labels_dir, city)
        for image_name in sorted(os.listdir(city_images_path)):
            if image_name.endswith('_leftImg8bit.png'):
                image_path = os.path.join(city_images_path, image_name)
                label_name = image_name.replace('_leftImg8bit.png', '_gtFine_labelIds.png')
                label_path = os.path.join(city_labels_path, label_name)
                count += 1
                yield image_path, label_path
    print(f'Total images processed: {count}')

def create_dataset():
    dataset = tf.data.Dataset.from_generator(
        generator=file_paths_generator,
        output_types=(tf.string, tf.string),
        output_shapes=(tf.TensorShape([]), tf.TensorShape([])))

    dataset = dataset.map(lambda x, y: preprocess_image(x, y))
    return dataset.repeat()

dataset = create_dataset().batch(8).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

try:
    for batch, (images, labels) in enumerate(dataset.take(2)):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f'Sample Image from Batch {batch+1}')
        plt.imshow(images[0].numpy().astype('uint8'))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f'Sample Label from Batch {batch+1}')
        plt.imshow(labels[0].numpy().squeeze(), cmap='gray')
        plt.axis('off')
        plt.show()
except tf.errors.OutOfRangeError:
    print("Attempted to access beyond the available data.")

In [ ]:
import os
import numpy as np

data_dir = '/home/maith/Desktop/cityscapes'
train_labels_dir = os.path.join(data_dir, 'gtFine/train')

def analyze_dataset(train_labels_dir):
    class_ids = []
    for city in sorted(os.listdir(train_labels_dir)):
        city_labels_path = os.path.join(train_labels_dir, city)
        for label_file in sorted(os.listdir(city_labels_path)):
            if label_file.endswith('_labelIds.png'):
                label_path = os.path.join(city_labels_path, label_file)
                label = tf.io.read_file(label_path)
                label = tf.image.decode_png(label, channels=1)
                unique_ids = np.unique(label.numpy())
                class_ids.extend(unique_ids)
    
    unique_class_ids = np.unique(class_ids)
    return unique_class_ids

unique_class_ids = analyze_dataset(train_labels_dir)
print(f"Unique class IDs in the dataset: {unique_class_ids}")
print(f"Total number of classes: {len(unique_class_ids)}")

In [ ]:
from tensorflow.keras.layers import Layer

class CustomTileLayer(Layer):
    def __init__(self, multiples, **kwargs):
        super(CustomTileLayer, self).__init__(**kwargs)
        self.multiples = multiples

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        outputs = inputs
        for axis, multiple in enumerate(self.multiples):
            if multiple > 1:
                outputs = tf.expand_dims(outputs, axis)
                tile_multiples = [1] * outputs.shape.rank
                tile_multiples[axis] = multiple
                outputs = tf.tile(outputs, tile_multiples)
        output_shape = [input_shape[i] * self.multiples[i] for i in range(len(self.multiples))]
        return tf.reshape(outputs, tf.concat([[input_shape[0]], output_shape[1:]], axis=0))

    def get_config(self):
        config = super(CustomTileLayer, self).get_config()
        config.update({
            'multiples': self.multiples,
        })
        return config

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, UpSampling2D, concatenate, Dropout

def conv_block(input_tensor, num_filters, dropout_rate=0.1):
    x = Conv2D(num_filters, (3, 3), padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def decoder_block(input_tensor, concat_tensor, num_filters):
    x = UpSampling2D((2, 2))(input_tensor)
    x = concatenate([x, concat_tensor], axis=-1)
    x = conv_block(x, num_filters)
    return x

def complex_unet_mobilenetv2(input_shape=(224, 224, 3), num_classes=34):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, alpha=1.0)
    base_model.trainable = True

    inputs = base_model.input
    
    f1 = base_model.get_layer("block_1_expand_relu").output
    f2 = base_model.get_layer("block_3_expand_relu").output
    f3 = base_model.get_layer("block_6_expand_relu").output
    f4 = base_model.get_layer("block_13_expand_relu").output
    f5 = base_model.get_layer("block_16_project").output
    
    b = conv_block(f5, 512)

    d1 = decoder_block(b, f4, 256)
    d2 = decoder_block(d1, f3, 128)
    d3 = decoder_block(d2, f2, 64)
    d4 = decoder_block(d3, f1, 32)
    d5 = UpSampling2D((2, 2))(d4)
    
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(d5)

    model = Model(inputs=inputs, outputs=outputs)
    return model

model = complex_unet_mobilenetv2()
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def compile_model(model):
    model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    )

compile_model(model)

print("Model compiled successfully with Adam optimizer, Sparse Categorical Crossentropy loss, and accuracy & MeanIoU metrics.")

In [ ]:
import tensorflow as tf

def load_and_preprocess_image(image_path, label_path, target_size=(224, 224)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label, axis=-1)

    return image, label

def prepare_dataset(image_dir, label_dir, batch_size=4):
    image_paths = [os.path.join(image_dir, city, f) for city in os.listdir(image_dir) for f in os.listdir(os.path.join(image_dir, city)) if f.endswith('_leftImg8bit.png')]
    label_paths = [p.replace('_leftImg8bit.png', '_gtFine_labelIds.png').replace('leftImg8bit', 'gtFine') for p in image_paths]

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

train_image_dir = os.path.join(data_dir, 'leftImg8bit/train')
train_label_dir = os.path.join(data_dir, 'gtFine/train')
val_image_dir = os.path.join(data_dir, 'leftImg8bit/val')
val_label_dir = os.path.join(data_dir, 'gtFine/val')

train_dataset = prepare_dataset(train_image_dir, train_label_dir)
val_dataset = prepare_dataset(val_image_dir, val_label_dir)

print("Training and validation datasets are ready.")

In [ ]:
# def check_dataset_shapes(dataset):
#     for images, labels in dataset.take(1):
#         print("Images shape:", images.shape)
#         print("Labels shape:", labels.shape)

# check_dataset_shapes(train_dataset)
# check_dataset_shapes(val_dataset)

In [ ]:
# from tensorflow.keras.layers import Conv2D

# def check_model_output(model, input_shape=(1, 256, 512, 3)):
#     test_input = tf.random.normal(input_shape)
#     test_output = model(test_input)
#     print("Test output shape:", test_output.shape)

# check_model_output(model)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

num_train_images = sum([len(files) for r, d, files in os.walk(train_image_dir)])
num_val_images = sum([len(files) for r, d, files in os.walk(val_image_dir)])

train_steps_per_epoch = num_train_images // 32
val_steps_per_epoch = num_val_images // 32

checkpoint_callback = ModelCheckpoint(
    os.path.join('/home/maith/Desktop/practical_2/cityscapes/', 'best_model.h5'), 
    monitor='val_loss', 
    save_best_only=True, 
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    steps_per_epoch=train_steps_per_epoch,
    validation_steps=val_steps_per_epoch,
    callbacks=[checkpoint_callback, early_stopping_callback],
    verbose=1
)

print("Training complete.")

In [ ]:
model.save('final_model.h5')
print("Model saved as final_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model saved as model.tflite")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

true_labels = []
pred_labels = []

for images, labels in val_dataset.take(10):
    preds = model.predict(images)
    preds = np.argmax(preds, axis=-1)
    true_labels.extend(labels.numpy().flatten())
    pred_labels.extend(preds.flatten())

true_labels = np.array(true_labels)
pred_labels = np.array(pred_labels)

cm = confusion_matrix(true_labels, pred_labels, labels=unique_class_ids)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=unique_class_ids, yticklabels=unique_class_ids)
plt.title('Normalized Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def load_and_preprocess_image(image_path, label_path, target_size=(256, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label, axis=-1)

    return image, label

# Paths to test images and labels
test_images_dir = os.path.join(data_dir, 'leftImg8bit/test')
test_labels_dir = os.path.join(data_dir, 'gtFine/test')

test_image_paths = [os.path.join(test_images_dir, city, f) for city in os.listdir(test_images_dir) for f in os.listdir(os.path.join(test_images_dir, city)) if f.endswith('_leftImg8bit.png')]
test_label_paths = [p.replace('_leftImg8bit.png', '_gtFine_labelIds.png').replace('leftImg8bit', 'gtFine') for p in test_image_paths]

index = 5
test_image_path = test_image_paths[index]
test_label_path = test_label_paths[index]

test_image, test_label = load_and_preprocess_image(test_image_path, test_label_path)

test_image_batch = tf.expand_dims(test_image, axis=0)
predicted_annotation = model.predict(test_image_batch)
predicted_annotation = np.argmax(predicted_annotation, axis=-1).squeeze()

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title('Test Image')
plt.imshow(test_image.numpy().astype('uint8'))
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('True Annotation')
plt.imshow(test_label.numpy(), cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Predicted Annotation')
plt.imshow(predicted_annotation, cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Define the function to load and preprocess a single image and its annotation
def load_and_preprocess_image(image_path, label_path, target_size=(256, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label, axis=-1)

    return image, label

# Paths to test images and labels
test_images_dir = os.path.join(data_dir, 'leftImg8bit/test')
test_labels_dir = os.path.join(data_dir, 'gtFine/test')

# Get a list of all test images and labels
test_image_paths = [os.path.join(test_images_dir, city, f) for city in os.listdir(test_images_dir) for f in os.listdir(os.path.join(test_images_dir, city)) if f.endswith('_leftImg8bit.png')]
test_label_paths = [p.replace('_leftImg8bit.png', '_gtFine_labelIds.png').replace('leftImg8bit', 'gtFine') for p in test_image_paths]

# Select 5 test images
indices = [0, 1, 2, 3, 4]  # Change indices as needed

plt.figure(figsize=(15, 10))

for i, index in enumerate(indices):
    # Load and preprocess a single test image and label
    test_image_path = test_image_paths[index]
    test_label_path = test_label_paths[index]

    test_image, test_label = load_and_preprocess_image(test_image_path, test_label_path)

    # Predict the annotation for the test image
    test_image_batch = tf.expand_dims(test_image, axis=0)  # Add batch dimension
    predicted_annotation = model.predict(test_image_batch)
    predicted_annotation = np.argmax(predicted_annotation, axis=-1).squeeze()

    # Display the test image, true annotation, and predicted annotation
    plt.subplot(5, 3, i*3 + 1)
    plt.title(f'Test Image {i+1}')
    plt.imshow(test_image.numpy().astype('uint8'))
    plt.axis('off')

    plt.subplot(5, 3, i*3 + 2)
    plt.title(f'True Annotation {i+1}')
    plt.imshow(test_label.numpy(), cmap='nipy_spectral')
    plt.axis('off')

    plt.subplot(5, 3, i*3 + 3)
    plt.title(f'Predicted Annotation {i+1}')
    plt.imshow(predicted_annotation, cmap='nipy_spectral')
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

cityscapes_colors = {
    0: (128, 64, 128),
    1: (244, 35, 232),
    2: (70, 70, 70),
    3: (102, 102, 156),
    4: (190, 153, 153),
    5: (153, 153, 153),
    6: (250, 170, 30),
    7: (220, 220, 0),
    8: (107, 142, 35),
    9: (152, 251, 152),
    10: (70, 130, 180),
    11: (220, 20, 60),
    12: (255, 0, 0),
    13: (0, 0, 142),
    14: (0, 0, 70),
    15: (0, 60, 100),
    16: (0, 80, 100),
    17: (0, 0, 230),
    18: (119, 11, 32),
    19: (0, 0, 0),
    20: (0, 0, 0),
    21: (0, 0, 0),
    22: (0, 0, 0),
    23: (0, 0, 70),
    24: (0, 60, 100),
    25: (0, 80, 100),
    26: (0, 0, 230),
    27: (119, 11, 32),
    28: (70, 70, 70),
    29: (102, 102, 156),
    30: (190, 153, 153),
    31: (153, 153, 153),
    32: (250, 170, 30),
    33: (220, 220, 0),
}

def create_color_mask(mask):
    color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for class_id, color in cityscapes_colors.items():
        color_mask[mask == class_id] = color
    return color_mask

def visualize_predictions(model, test_images, num_images=5):
    fig, axes = plt.subplots(num_images, 3, figsize=(20, num_images * 5))
    random_indices = random.sample(range(len(test_images)), num_images)

    for i, idx in enumerate(random_indices):
        test_image_path = test_images[idx]
        test_image = tf.image.decode_image(tf.io.read_file(test_image_path))
        test_image = tf.image.resize(test_image, (256, 512))
        test_image = tf.expand_dims(test_image, axis=0)
        
        pred_mask = model.predict(test_image)
        pred_mask = np.argmax(pred_mask, axis=-1)[0]
        pred_color_mask = create_color_mask(pred_mask)

        axes[i, 0].imshow(plt.imread(test_image_path))
        axes[i, 0].set_title(f"Test Image {i + 1}")
        axes[i, 0].axis('off')

        axes[i, 1].imshow(pred_mask, cmap='gray')
        axes[i, 1].set_title(f"Predicted Annotation {i + 1}")
        axes[i, 1].axis('off')

        axes[i, 2].imshow(pred_color_mask)
        axes[i, 2].set_title(f"Colorized Prediction {i + 1}")
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

test_images_dir = '/home/maith/Desktop/practical_2/cityscapes/leftImg8bit/test'
test_image_paths = [os.path.join(test_images_dir, city, f) for city in os.listdir(test_images_dir) for f in os.listdir(os.path.join(test_images_dir, city)) if f.endswith('_leftImg8bit.png')]

visualize_predictions(model, test_image_paths)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

def load_and_preprocess_image(image_path, label_path, target_size=(256, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=1)
    label = tf.image.resize(label, target_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    label = tf.squeeze(label, axis=-1)

    return image, label

def overlay_annotation_on_image(image, annotation, alpha=0.5):
    color_annotation = tf.keras.utils.to_categorical(annotation, num_classes=34) 
    color_annotation = np.argmax(color_annotation, axis=-1)
    
    np.random.seed(42)
    colors = np.random.randint(0, 255, size=(34, 3), dtype=int)
    
    overlay = np.zeros_like(image)
    for class_id in range(34):
        overlay[color_annotation == class_id] = colors[class_id]
    
    overlay_image = (alpha * image + (1 - alpha) * overlay).astype(np.uint8)
    return overlay_image

test_images_dir = os.path.join(data_dir, 'leftImg8bit/test')
test_labels_dir = os.path.join(data_dir, 'gtFine/test')

test_image_paths = [os.path.join(test_images_dir, city, f) for city in os.listdir(test_images_dir) for f in os.listdir(os.path.join(test_images_dir, city)) if f.endswith('_leftImg8bit.png')]
test_label_paths = [p.replace('_leftImg8bit.png', '_gtFine_labelIds.png').replace('leftImg8bit', 'gtFine') for p in test_image_paths]

num_images_to_display = 20

plt.figure(figsize=(15, 10))

for i in range(num_images_to_display):
    test_image_path = test_image_paths[i]
    test_label_path = test_label_paths[i]

    test_image, test_label = load_and_preprocess_image(test_image_path, test_label_path)

    test_image_batch = tf.expand_dims(test_image, axis=0)
    predicted_annotation = model.predict(test_image_batch)
    predicted_annotation = np.argmax(predicted_annotation, axis=-1).squeeze()

    overlayed_image = overlay_annotation_on_image(test_image.numpy().astype('uint8'), predicted_annotation)

    plt.subplot(num_images_to_display // 5, 5, i + 1)
    plt.imshow(overlayed_image)
    plt.axis('off')
    plt.title(f'Test Image {i+1}')

plt.tight_layout()
plt.show()

In [ ]:
# Convert to quantized .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

def representative_data_gen():
    for input_value, _ in val_dataset.take(100):
        yield [input_value]

converter.representative_dataset = representative_data_gen
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)
print(f"Model saved as model_quantized.tflite")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

custom_objects = {'CustomTileLayer': CustomTileLayer}

model = load_model('/home/maith/Desktop/cityscapes/accure_models/final_model.h5', custom_objects=custom_objects)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

def representative_data_gen():
    for input_value, _ in val_dataset.take(10):
        yield [input_value]

converter.representative_dataset = representative_data_gen
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

with open('model_quantized_no_tile.tflite', 'wb') as f:
    f.write(tflite_model)
print(f"Model saved as model_quantized_no_tile.tflite")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

custom_objects = {'CustomTileLayer': CustomTileLayer}

# Load the original Keras model
model = load_model('/home/maith/Desktop/cityscapes/accure_models/best_model.h5', custom_objects=custom_objects)

# Convert the model to a TensorFlow Lite model with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Representative dataset function
def representative_data_gen():
    for input_value, _ in val_dataset.take(10):
        yield [input_value]

converter.representative_dataset = representative_data_gen
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

quantized_tflite_model = converter.convert()

# Save the quantized model
quantized_model_path = '/home/maith/Desktop/cityscapes/accure_models/model_quantized_3.tflite'
with open(quantized_model_path, 'wb') as f:
    f.write(quantized_tflite_model)

print(f"Quantized model saved as {quantized_model_path}")

In [ ]:
from tensorflow.keras.models import load_model

custom_objects = {'CustomTileLayer': CustomTileLayer}

model = load_model('/home/maith/Desktop/practical_2/cityscapes/final_model.h5', custom_objects=custom_objects)